# Holder Wallet Conformance Flows (v3) - CTWalletCrossPreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from eudi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'ebsitests'

key_did = KeyDid(seed=crypto_seed)
# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9Kbowo1MYpCJwNfzfFggXuWxdPAdAWhkp9XskU2cir7r21AxqN2TMvMTUS1FmaNWLm2esa8FLwZ35i5mRzUygDQdZRS1gC42Am9Doyh68Hp1Ksap9e9xzebDYFhhLgXTksgnG


## 1.1 Initiate Credential Issuance

In [3]:
from eudi_wallet.util import parse_query_string_parameters_from_url
from eudi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletCrossPreAuthorised
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
# qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fissuer-mock%2Foffers%2F674c272c-4d21-47d7-9190-2fd26ebc56bb"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[23:37:19] Credential offer URI:                                                                     ]8;id=556129;file:///tmp/ipykernel_70412/61612052.py\61612052.py]8;;\:]8;id=520514;file:///tmp/ipykernel_70412/61612052.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/3a09401e-2819-4539-9e5a               
           -7d9635c95056                                                                                           

{'credential_issuer': 'https://api-conformance.ebsi.eu/conformance/v3/issuer-mock', 'credentials': [{'format': 'jwt_vc', 'types': ['VerifiableCredential', 'VerifiableAttestation', 'CTWalletCrossPreAuthorised'], 'trust_framework': {'name': 'ebsi', 'type': 'Accreditation', 'uri': 'TIR link towards accreditation'}}], 'grants': {'urn:ietf:params:oauth:grant-type:pre-authorized_code': {'pre-authorized_code': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlRNdk1UVVMxRm1hTldMbTJlc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emViRFlGaGhMZ1hUa3NnbkciLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlma

           Credential offer:                                                                         ]8;id=31985;file:///tmp/ipykernel_70412/61612052.py\61612052.py]8;;\:]8;id=705647;file:///tmp/ipykernel_70412/61612052.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossPreAuthorised'                                                            
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code':                                                                      
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZq               
           cFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW               
           95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlR               
           Ndk1UVVMxRm1hTldMbTJlc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emVi               
           RFlGaGhMZ1hUa3NnbkciLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhb               
           CIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2               
           NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZ               
           pYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJpYXQiOjE2OTA5OTk2Mzks               
           ImV4cCI6MTY5MDk5OTkzOSwiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3JtYW5jZ               
           S92My9pc3N1ZXItbW9jayIsImF1ZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2               
           UvdjMvYXV0aC1tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1V               
           aM2V5cWh0MWo5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVT               
           MUZtYU5XTG0yZXNhOEZMd1ozNWk1bVJ6VXl

[23:37:20] Credential issuer configuration:                                                          ]8;id=12442;file:///tmp/ipykernel_70412/61612052.py\61612052.py]8;;\:]8;id=654173;file:///tmp/ipykernel_70412/61612052.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/crede               
           ntial',                                                                                                 
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-m               
           ock/credential_deferred',                                                                               
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': {   

[23:37:21] Authorization server configuration:                                                       ]8;id=161946;file:///tmp/ipykernel_70412/61612052.py\61612052.py]8;;\:]8;id=323380;file:///tmp/ipykernel_70412/61612052.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/auth               
           orize',                                                                                                 
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Exchange code for access token

In [4]:
from eudi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
user_pin = "4695"
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, user_pin, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

{'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImFlYjViN2ZkLTVmYjgtNGE1Ny1hNDc1LTZhNDA0ODdiYTEyNCIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjoiOWRkZGEwZDAtNzg5NS00Mjg1LWIzNzAtNDgwNDlhODc4ZDBhIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCwiY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0yZXNhOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25HIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwiYXVkIjpbImh0dHBzOi8vYXBpLWN

           Access token response:                                                                   ]8;id=640590;file:///tmp/ipykernel_70412/3067923722.py\3067923722.py]8;;\:]8;id=487120;file:///tmp/ipykernel_70412/3067923722.py#6\6]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYW                
           VZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImFlYjViN2ZkLTVmYjgtNGE1Ny1hNDc1LTZhNDA                
           0ODdiYTEyNCIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGV                
           udGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnN                
           pLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjo                
           iOWRkZGEwZDAtNzg5NS00Mjg1LWIzNzAtNDgwNDlhODc4ZDBhIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCw                
           iY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0                
           yZXNhOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25                
           HIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2N                
           rIiwiYXVkIjpbImh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvaXNzdWVyLW1                
           vY2siXSwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0                
           yZXNhOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25                
           HIiwiaWF0IjoxNjkwOTk5NjQxLCJleHAiOjE2OTEwODYwNDF9.jgnMeEWrtuyJQERyjCgBDtgVwmC8KZE7n3i6OQ                
           c9LEz5mVPjZKSU_dH4Tbm4PePZZxLFaNKpKP7K7kcLrW3Fmg',                                                      
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUW                
           lGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImVhOTM1YTlmLWY0OGEtNGE4Zi05YTEwLWFkMzEyOTY                
           yNGNhMyIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1                
           tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0                
           yZXNhOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25                
           HIiwiYXVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J                
           vd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0yZXN                
           hOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25HIiw                
           iaWF0IjoxNjkwOTk5NjQxLCJleHAiOjE2OTA5OTk3NjF9.wYBkJbwwgChOVZBWnrcTtKd7Y4D0DqgkheC4oph-yV                
           WZLQal915JJJpoGcdVRtCf9LcMe6upCRCIIET-FgSPMg',                                                          
               c_nonce='9ddda0d0-7895-4285-b370-48049a878d0a',                                                     
               c_nonce_expires_in=86400,  

## 1.3 Request credential (cross device)

In [5]:
from eudi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
print("Credential response: ", credential_response)

           Credential request JWT:                                                                   ]8;id=498622;file:///tmp/ipykernel_70412/621865142.py\621865142.py]8;;\:]8;id=926453;file:///tmp/ipykernel_70412/621865142.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUz               
           FGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0RRZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaEx               
           nWFRrc2duRyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2Jvd28xTVlwQ0p3               
           TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0yZXNhOEZMd1ozNWk1b               
           VJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25HIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjkxMDg2MDQxLCJpYXQiOjE2OTA5OTk2NDEsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2               
           dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUzFGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0R               
           RZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaExnWFRrc2duRyIsIm5vbmNlIjoiOWRkZGEwZDAt               
           Nzg5NS00Mjg1LWIzNzAtNDgwNDlhODc4ZDBhIn0.hxC8aAMS6ej2d-NG_pDRf9aFoPL8GZi6c_ZecyyfMsmjN3dMj               
           mCcKbku6xo97wPQjEvsjKSIkRe7tcPdwsEGCg                                                                   

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2FmOTc2NWJhLTViOTMtNDNkZi1hOTViLTMwMjRlODBkZTIwYiIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUzFGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0RRZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaExnWFRrc2duRyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjkwOTk5NjQyLCJleHAiOjE2OTEwODYwNDIsImlhdCI6MTY5MDk5OTY0MiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2FmOTc2NWJhLTViOTMtNDNkZi1hOTViLTMwMjRlODBkZTIwYiIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3NzUHJlQXV0aG9yaXNlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V